# Deep Convolution GAN (DCGAN) using FACES Dataset

**Objective**: Implement Deep Convolutional GAN using Faces Dataset.

Dataset Location: https://www.kaggle.com/gasgallo/faces-data-new

Paper:  
Radford, A., Metz, L., & Chintala, S. (2015). [Unsupervised representation learning with deep convolutional generative adversarial networks.](https://arxiv.org/pdf/1511.06434.pdf)


## Import Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!cp '/content/drive/My Drive/App/DCGAN/faces-data-new.zip' .
#!cp '/content/drive/My Drive/App/DCGAN/images_arr.64.npy' .

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import sys

## Configurations

In [0]:
IMAGE_DATA_FILE = '/content/images_arr.64.npy'

# image details
img_rows = 64
img_cols = 64
channels = 3
image_shape = (img_rows, img_cols, channels)

## Latent Space (vectors) dimension
latent_dim = 100

# generated image resolution
# 1 => 32 pixels, 2 = 32*2 = 64, 3=> 32*3 => 96 etc
gen_image_res = 2

In [0]:
images_arr = np.load(IMAGE_DATA_FILE)

In [6]:
images_arr.shape

(7864, 64, 64, 3)

## Normalize the input data -1 to 1

In [0]:
X_train = images_arr

In [0]:
# rescale the images : -1 to 1 instead of 0 to 255

X_train = X_train / 127.5 -1

## Build DCGAN

In [9]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
?BatchNormalization

### Discriminator

In [0]:
# Discrimator
#
# Use Conv2D to identify image elements
# Use Dropout to generalize better.
# Use BatchNormalization - Normalize the activations of the prev layer at each batch.
#     -Moemntum for moving mean and moving variance.
# Use LeakyReLU for activation
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    input_image = Input(shape=image_shape)

    validity = model(input_image)

    return Model(input_image, validity)
  

## Generator

In [0]:
# Generator :
#    latent vector(space) -> image generation.
#    in our case vector of size 100 -> image of 64x64
# 
def build_generator(latent_dim, channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=latent_dim))
    model.add(Reshape((4,4,256)))

    #Four layers of upsampling, convolution, batch normalization and activation.
    # Further more blocks can be repeated for higher resolution.
    #
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    for i in range(gen_image_res):
      model.add(UpSampling2D())
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    input = Input(shape=(latent_dim,))
    generated_image = model(input)

    return Model(input,generated_image)

## Build DCGAN

In [13]:
optimizer = Adam(1.5e-4,0.5) # learning rate and momentum adjusted from paper

# build discriminator
discriminator = build_discriminator(image_shape)
discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

W0802 08:35:58.232087 140720216610688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 08:35:58.233667 140720216610688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 08:35:58.240148 140720216610688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 08:35:58.254800 140720216610688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0802 08:35:58.263603 

In [14]:
# build generator
generator = build_generator(latent_dim,channels)

# build the stacked model (combined model)
random_input = Input(shape=(latent_dim,))

generated_image = generator(random_input)

# freeze the discrimator
discriminator.trainable = False

validity = discriminator(generated_image)

# use functional Model API
combined = Model(random_input,validity)

combined.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=["accuracy"])

W0802 08:35:59.878432 140720216610688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



In [15]:
combined.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 64, 64, 3)         2043011   
_________________________________________________________________
model_1 (Model)              (None, 1)                 1613889   
Total params: 3,656,900
Trainable params: 2,041,475
Non-trainable params: 1,615,425
_________________________________________________________________


## Training

In [0]:
epochs = 10000
batch_size = 32
sample_interval = 200

In [17]:
!mkdir /content/gen_images2

mkdir: cannot create directory ‘/content/gen_images2’: File exists


In [0]:
#sample images
def sample_images(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :,:,0])
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("/content/gen_images2/%d.png" % epoch)
  plt.close()

In [19]:
# build Adversarial ground truths for discriminator and generator
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Train both the models.
#
for epoch in range(epochs):

  #--------------------
  # train discriminator
  #--------------------
  # Select a random batch of images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]

  # Generate a batch of new images, using generator
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = generator.predict(noise)

  # train generator using both real and fake images.
  d_loss_real = discriminator.train_on_batch(imgs, valid)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  #-------------
  # train generator
  #-----------------
  noise = np.random.normal(0, 1, (batch_size, latent_dim))

  # Train the generator (to have the discriminator label samples as valid)
  g_loss = combined.train_on_batch(noise, valid)

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
    # Plot the progress
    print ("Epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % 
                          (epoch, d_loss[0], 100*d_loss[1], g_loss[0], 100*g_loss[1]))
    sample_images(epoch)
    

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0 [D loss: 1.756560, acc.: 6.25%] [G loss: 0.541157, acc.: 78.12%]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 200 [D loss: 0.192398, acc.: 92.19%] [G loss: 5.316466, acc.: 0.00%]
Epoch 400 [D loss: 0.627317, acc.: 76.56%] [G loss: 2.559813, acc.: 12.50%]
Epoch 600 [D loss: 0.955609, acc.: 62.50%] [G loss: 1.763058, acc.: 18.75%]
Epoch 800 [D loss: 0.859079, acc.: 60.94%] [G loss: 1.764447, acc.: 28.12%]
Epoch 1000 [D loss: 0.848003, acc.: 57.81%] [G loss: 1.586489, acc.: 25.00%]
Epoch 1200 [D loss: 0.778090, acc.: 56.25%] [G loss: 2.418842, acc.: 12.50%]
Epoch 1400 [D loss: 0.453121, acc.: 82.81%] [G loss: 2.941964, acc.: 9.38%]
Epoch 1600 [D loss: 0.532333, acc.: 75.00%] [G loss: 2.772845, acc.: 0.00%]
Epoch 1800 [D loss: 0.437167, acc.: 79.69%] [G loss: 2.403053, acc.: 9.38%]
Epoch 2000 [D loss: 0.405635, acc.: 84.38%] [G loss: 3.270834, acc.: 6.25%]
Epoch 2200 [D loss: 0.552522, acc.: 71.88%] [G loss: 3.127317, acc.: 6.25%]
Epoch 2400 [D loss: 0.383389, acc.: 79.69%] [G loss: 3.123402, acc.: 0.00%]
Epoch 2600 [D loss: 0.525774, acc.: 75.00%] [G loss: 3.455653, acc.: 9.38%]
Epoch 2800 

In [21]:
!ls -l /content/gen_images2 | wc -l


51


In [0]:
!cp '/content/gen_images2/1000.png' '/content/drive/My Drive/App/DCGAN/gen_images'

In [0]:
!cp '/content/gen_images2/4000.png' '/content/drive/My Drive/App/DCGAN/gen_images'
!cp '/content/gen_images2/6000.png' '/content/drive/My Drive/App/DCGAN/gen_images'
!cp '/content/gen_images2/8000.png' '/content/drive/My Drive/App/DCGAN/gen_images'
!cp '/content/gen_images2/9000.png' '/content/drive/My Drive/App/DCGAN/gen_images'